# Quantiative Algorithm using Technical Indicators

Technical data obtained from TradingView
 

## The Data

In [1]:
import pandas as pd
import numpy as np

In [2]:
stock_data = pd.read_csv('SPY Original.csv')
stock_data = stock_data.drop('timestamp', axis=1)
stock_data = stock_data.replace(to_replace='None', value=np.nan).dropna()
stock_data = stock_data.astype(float)

In [3]:
x_data = stock_data.drop(['%'],axis=1)
y_val = stock_data['%']

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_data,y_val,test_size=0.3, shuffle=False)

### Scale the Data

In [5]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [6]:
X_train = pd.DataFrame(data=scaler.transform(X_train),columns = X_train.columns,index=X_train.index)
X_test = pd.DataFrame(data=scaler.transform(X_test),columns = X_test.columns,index=X_test.index)

In [7]:
X_train.head(2)

,open,high,low,close,vol,vol_20d_MA,BB_1,BB_2,BB_3,Ichimoku _1,...,OBV,ADX,MACD_1,MACD_2,MACD_3,Stoch_RSI_1,Stoch_RSI_2,Klinger_Oscillator_1,Klinger_Oscillator_2,CMF
66,0.009904,0.006333,0.010039,0.006107,0.000119,0.000095,0.001777,0.004166,0.000547,0.0,...,0.199090,0.338495,0.608831,0.687297,0.700713,0.919842,0.776481,0.580295,0.515759,0.442423
67,0.010455,0.004681,0.009481,0.004719,0.000057,0.000098,0.001820,0.004212,0.000586,0.0,...,0.199085,0.327429,0.610870,0.689399,0.702162,0.937505,0.877542,0.580279,0.515774,0.418237


## Keras Modeling

In [8]:
from keras.layers import concatenate, LSTM, Flatten, Input, Dense, Conv2D, Flatten, Conv1D, GlobalAveragePooling1D, MaxPooling1D
from keras.models import Model
from keras.utils import plot_model

Using TensorFlow backend.


In [9]:
#Global Setup
time_period = len(X_train)

In [10]:
#Note: Output Shape is (Input Height - Filter Height + 1 (if strides=1))

#Trend Indicators

#MACD Setup
macd_df = pd.concat([X_train['MACD_1'], X_train['MACD_2'], X_train['MACD_3']], axis=1)
macd_outputs = len(macd_df.columns)
#Ichimoku Setup
ichimoku_df = pd.concat([X_train['Ichimoku _1'], X_train['Ichimoku_2'], X_train['Ichimoku_3'], X_train['Ichimoku_4'], X_train['Ichimoku_5']], axis=1)
ichimoku_outputs = len(ichimoku_df.columns)
#SAR Setup
sar_df = pd.concat([X_train['SAR']], axis=1)
sar_outputs = len(sar_df.columns)

#MACD
#Conv1D Shape [# Filters, Filter Height]
visible_macd = Input(shape=(time_period,macd_outputs))
conv_macd = Conv1D(100, 5, name='3MACD_Conv', activation='relu')(visible_macd)
pool_macd = MaxPooling1D(3)(conv_macd)
flat_macd = Flatten()(pool_macd)

#Ichimoku
#Conv1D Shape [# Filters, Filter Height]
visible_ichimoku = Input(shape=(time_period,ichimoku_outputs))
conv_ichimoku = Conv1D(100, 5, name='5Ichimoku_Conv', activation='relu')(visible_ichimoku)
pool_ichimoku = MaxPooling1D(3)(conv_ichimoku)
flat_ichimoku = Flatten()(pool_ichimoku)

#SAR
#Conv1D Shape [# Filters, Filter Height]
visible_sar = Input(shape=(time_period,sar_outputs))
pool_sar = MaxPooling1D(3)(visible_sar)
flat_sar = Flatten()(pool_sar)

#All Trend Indicators
merge_trend = concatenate([flat_macd, flat_ichimoku, flat_sar])
dense_trend = Dense(100, activation='relu')(merge_trend)

In [11]:
#Volatility Indicators

#Bollinger Bands
bollinger_df = pd.concat([X_train['BB_1'], X_train['BB_2'], X_train['BB_3']], axis=1)
bollinger_outputs = len(bollinger_df.columns)

#Conv1D Shape [# Filters, Filter Height]
visible_bollinger = Input(shape=(time_period,bollinger_outputs))
conv_bollinger = Conv1D(100, 5, name='3Bollinger_Conv', activation='relu')(visible_bollinger)
pool_bollinger = MaxPooling1D(3)(conv_bollinger)
flat_bollinger = Flatten()(pool_bollinger)

#All Volatility Indicators
# merge_volatility = concatenate([flat_bollinger])
dense_volatility = Dense(100, activation='relu')(flat_bollinger)

In [12]:
#Momentum Indicators

#stochRSI Setup
stochRSI_df = pd.concat([X_train['Stoch_RSI_1'], X_train['Stoch_RSI_2']], axis=1)
stochRSI_outputs = len(stochRSI_df.columns)
#adx Setup
adx_df = pd.concat([X_train['ADX']], axis=1)
adx_outputs = len(adx_df.columns)

#stochRSI
#Conv1D Shape [# Filters, Filter Height]
visible_stochRSI = Input(shape=(time_period,stochRSI_outputs))
conv_stochRSI = Conv1D(100, 5, name='2StochRSI_Conv', activation='relu')(visible_stochRSI)
pool_stochRSI = MaxPooling1D(3)(conv_stochRSI)
flat_stochRSI = Flatten()(pool_stochRSI)

#adx
#Conv1D Shape [# Filters, Filter Height]
visible_adx = Input(shape=(time_period,adx_outputs))
pool_adx = MaxPooling1D(3)(visible_adx)
flat_adx = Flatten()(pool_adx)

#All Momentum Indicators
merge_momentum = concatenate([flat_stochRSI, flat_adx])
dense_momentum = Dense(100, activation='relu')(merge_momentum)

In [13]:
#Volume Indicators

#KlingerVol Setup
klingerVol_df = pd.concat([X_train['Klinger_Oscillator_1'], X_train['Klinger_Oscillator_2']], axis=1)
klingerVol_outputs = len(klingerVol_df.columns)
#OBV Setup
obv_df = pd.concat([X_train['OBV']], axis=1)
obv_outputs = len(obv_df.columns)
#CMF Setup
cmf_df = pd.concat([X_train['CMF']], axis=1)
cmf_outputs = len(cmf_df.columns)

#KlingerVol
#Conv1D Shape [# Filters, Filter Height]
visible_klingerVol = Input(shape=(time_period,klingerVol_outputs))
conv_klingerVol = Conv1D(100, 5, name='2KlingerVol_Conv', activation='relu')(visible_klingerVol)
pool_klingerVol = MaxPooling1D(3)(conv_klingerVol)
flat_klingerVol = Flatten()(pool_klingerVol)

#OBV
#Conv1D Shape [# Filters, Filter Height]
visible_obv = Input(shape=(time_period,obv_outputs))
pool_obv = MaxPooling1D(3)(visible_obv)
flat_obv = Flatten()(pool_obv)

#CMF
#Conv1D Shape [# Filters, Filter Height]
visible_cmf = Input(shape=(time_period,cmf_outputs))
pool_cmf = MaxPooling1D(3)(visible_cmf)
flat_cmf = Flatten()(pool_cmf)

#All Volume Indicators
merge_volume = concatenate([flat_klingerVol, flat_obv, flat_cmf])
dense_volume = Dense(100, activation='relu')(merge_volume)

In [15]:
#All Indicators
merge_all = concatenate([dense_trend, dense_volatility, dense_momentum, dense_volume])
#Final Output
output = Dense(100, activation='relu')(merge_all)
#Create Model
visibles_list = [visible_macd, visible_ichimoku, visible_sar, visible_bollinger, visible_stochRSI, visible_adx, visible_klingerVol, visible_obv, visible_cmf]
model = Model(inputs=visibles_list, outputs=output)
#Print Model Text Summary
print(model.summary())
#Save Model Graph
plot_model(model, to_file='./model.png', show_shapes=True)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 4564, 3)      0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 4564, 5)      0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            (None, 4564, 2)      0                                            
__________________________________________________________________________________________________
input_7 (InputLayer)            (None, 4564, 2)      0                                            
__________________________________________________________________________________________________
3MACD_Conv

In [28]:
X_input_list = [macd_df.values, ichimoku_df.values, sar_df.values, bollinger_df.values, stochRSI_df.values, adx_df.values, klingerVol_df.values, obv_df.values, cmf_df.values]

for i in X_input_list:
    i = i.reshape([-1,time_period,i.shape[1]])
    
value = macd_df.values
value = value.reshape([-1,time_period, 3])
value.shape

(1, 4564, 3)

In [17]:
from sklearn.metrics import mean_squared_error

#Model Creation & Usage
model.compile(loss='mean_squared_error', optimizer='adam')

model.fit(x=X_input_list, y=y_train, epochs=100, batch_size=1, verbose=2)

ValueError: Error when checking input: expected input_1 to have 3 dimensions, but got array with shape (4564, 3)

# We Did It?